# Data Collection

In order to best answer the question **“Is London really as rainy as the movies make it out to be?”** several variables need to be considered: the time period, other cities to compare against London, how to measure and define raininess and as such which Open Meteo variables will be used. 

### The time period
To explore this question in depth I am going to focus on a very historical time period (1st January 1950 - 31st December 1970) and a more recent time period (1st January 2000 - 31st December 2020)
### Other cities to compare against London
To assess whether London is as rainy as movies suggest, a broad sample of global cities should be used in the comparison. This will enhance the reliability of findings, ensuring different climates are accounted for. As such the cities I will investigate are:
 - Manchester, UK: offers a regional counterpoint to London, indicating whether the amount of rain in London is abnormal for England
 - Edinburgh, Scotland: provides another UK perspective, allowing the assessment of London's weather in comparison to the broader UK climate
 - Paris, France: located relatively close to London and at a similar latitude, which may offer a similar climate and allowing the comaprison between major European cities
 - Rome, Italy: offers a warmer Mediterranean climate to contrast London's apparent 'rainier' climate
 - Kyiv, Ukraine: a more eastern European city with a similar latitude to London
 - Seattle, Washington, USA: another city that possesses 'a rainy reputation', making it a potentially good benchmark for comparison
 - Toronto, Canada: a major city on the east coast of North America, adding to the geographic diversity and possessing a climate influenced by the Great Lakes
 - Phoenix, Arizona, USA: a drier North American desert climate for contrast, enabling the evaluation of London's weather on a global scale
 - Bogotá, Columbia: a tropical climate in South America, characterised by high humidity and significant rainfall providing a valuable perspective on London's rainfall
 - Cairo, Egypt: a desert city with minimal rain, highlighting extremes in rainfall levels
 - Cape Town, South Africa: another city in the Southern Hemisphere with distinct wet and dry seasons
 - Omsk, Russia: located in Siberia to offer another contrasting point with London
 - Mumbai, India: a tropical monsoon climate, with heavy rains to offer comparison with London
 - Tokyo, Japan: an eastern Asian city to provide another global counterpoint
 - Auckland, New Zealand: very southern city to provide another global counterpoint

In [4]:
import json 
import requests
from lets_plot import *
import pandas as pd

LetsPlot.setup_html()


In [3]:
from lets_plot import *
import pandas as pd

LetsPlot.setup_html()

def plot_city_map(city_data, point_size=1, point_colour='blue', title='City Locations', title_size=30):
    
    # Parameters:
    #- city_data: List of tuples containing (city_name, latitude, longitude).
    #- point_size: Size of the points on the map (default is 5).
    #- point_colour: Color of the points (default is 'blue').
    #- title: Title of the plot (default is 'City Locations').
    
    
    # Create a DataFrame from the provided list of tuples
    city_df = pd.DataFrame(city_data, columns=['city', 'latitude', 'longitude'])
    
    # Plotting the points on a map
    plot = (
        ggplot() +
        geom_livemap() +
        geom_point(aes(x='longitude', y='latitude'), size=point_size, colour=point_colour, show_legend=False, data=city_df) +
        ggtitle(title) +
        theme_minimal() + 
        theme(plot_title=element_text(size=title_size, hjust=0.5))
    )
    
    return plot

# Sample data as a list of tuples (city, latitude, longitude)
city_coords = [
    ("London", 51.5072, -0.1276),  
    ("Manchester", 53.4808, -2.2426),  
    ("Edinburgh", 55.9533, -3.1883),
    ("Cork", 51.8985, -8.4756),
    ("Paris", 48.8575, 2.3514),
    ("Rome", 41.8967, 12.4822),
    ("Seattle", 47.6061, -122.3328),    
    ("Bogota", 4.7110, -74.0721),  
    ("Cairo", 30.0444, 31.2357),
    ("Cape Town", -33.9221, 18.4231),
    ("Mumbai", 19.0760, 72.8777)
]

# Call the function to plot the city map
city_map_plot = plot_city_map(city_coords)
city_map_plot


As the map demonstrates, these locations will provide a wide range of counterpoints to compare London against.

### Measuring and defining raininess
Raininess is based upon the frequency and duration of the rain and the amount of precipiation. As such the variables in Open-Meteo to be used are:
- Precipitation Sum 
- Rain Sum 
- Precipitation Hours

## Data collection

## Collecting historical data

In [1]:

def get_historical_data(city_coords, start_date, end_date):
    base_historical_url = "https://archive-api.open-meteo.com/v1/archive"
    precipitation = {}
    
    for city, latitude, longitude in city_coords:
        params_lat_long = f"latitude={latitude}&longitude={longitude}"
        params_others = "&daily=precipitation_sum,rain_sum,precipitation_hours"  # Changed to daily parameters
        params_dates = f"&start_date={start_date}&end_date={end_date}"
        
        end_url = base_historical_url + '?' + params_lat_long + params_others + params_dates
        
        historical_response = requests.get(end_url)
        
        if historical_response.status_code == 200:
            historical_data = historical_response.json()
            # Extract the relevant precipitation data
            historical_precipitation = {
                "precipitation_sum": historical_data['daily']['precipitation_sum'],
                "rain_sum": historical_data['daily']['rain_sum'],
                "precipitation_hours": historical_data['daily']['precipitation_hours'],
            }
            # Store the historical data for each city
            precipitation[city] = historical_precipitation
        else:
            print(f"Error for {city}: {historical_response.status_code}")
            precipitation[city] = None  # Store None if there's an error for that city

    with open("precipitation_data_hist.json", "w") as file:
        json.dump(precipitation, file, indent=4) # saving the data to a json file
    
    return precipitation  # Return the temperatures dictionary after the loop

 

# City coordinates
city_coords = [
    ("London", 51.5072, -0.1276),  
    ("Manchester", 53.4808, -2.2426),  
    ("Edinburgh", 55.9533, -3.1883),
    ("Cork", 51.8985, -8.4756),
    ("Paris", 48.8575, 2.3514),
    ("Rome", 41.8967, 12.4822),
    ("Seattle", 47.6061, -122.3328),    
    ("Bogota", 4.7110, -74.0721),  
    ("Cairo", 30.0444, 31.2357),
    ("Cape Town", -33.9221, 18.4231),
    ("Mumbai", 19.0760, 72.8777)
]

# Call the function to get historical data for the cities
precipitation_hist = get_historical_data(city_coords, start_date="1970-01-01", end_date="1980-12-31")



### Collecting data for the more recent time period
Done seperately to ensure that there were not too many API requests in one day

In [1]:

def get_recent_data(city_coords, start_date, end_date):
    base_historical_url = "https://archive-api.open-meteo.com/v1/archive"
    precipitation = {}
    
    for city, latitude, longitude in city_coords:
        params_lat_long = f"latitude={latitude}&longitude={longitude}"
        params_others = "&daily=precipitation_sum,rain_sum,precipitation_hours"  # Changed to daily parameters
        params_dates = f"&start_date={start_date}&end_date={end_date}"
        
        end_url = base_historical_url + '?' + params_lat_long + params_others + params_dates
        
        recent_response = requests.get(end_url)
        
        if recent_response.status_code == 200:
            recent_data = recent_response.json()
            # Extract the relevant precipitation data
            recent_precipitation = {
                "precipitation_sum": recent_data['daily']['precipitation_sum'],
                "rain_sum": recent_data['daily']['rain_sum'],
                "precipitation_hours": recent_data['daily']['precipitation_hours'],
            }
            # Store the recent data for each city
            precipitation[city] = recent_precipitation
        else:
            print(f"Error for {city}: {recent_response.status_code}")
            precipitation[city] = None  # Store None if there's an error for that city

    with open("precipitation_data_rec.json", "w") as file:
        json.dump(precipitation, file, indent=4) # saving the data to a json file
    
    return precipitation  # Return the temperatures dictionary after the loop

 

# City coordinates
city_coords = [
   ("London", 51.5072, -0.1276),  
    ("Manchester", 53.4808, -2.2426),  
    ("Edinburgh", 55.9533, -3.1883),
    ("Cork", 51.8985, -8.4756),
    ("Paris", 48.8575, 2.3514),
    ("Rome", 41.8967, 12.4822),
    ("Seattle", 47.6061, -122.3328),    
    ("Bogota", 4.7110, -74.0721),  
    ("Cairo", 30.0444, 31.2357),
    ("Cape Town", -33.9221, 18.4231),
    ("Mumbai", 19.0760, 72.8777)
]

# Call the function to get historical data for the cities
precipitation_recent = get_recent_data(city_coords, start_date="2010-01-01", end_date="2020-12-31")



NameError: name 'pd' is not defined

#### Creating a dataframe from the JSON files

In [5]:

# Load JSON data from both files
with open("precipitation_data_hist.json") as precip_hist, open("precipitation_data_rec.json") as precip_rec:
    precipitation_data_hist = json.load(precip_hist)
    precipitation_data_rec = json.load(precip_rec)

# Create a function to transform JSON data into a tidy format
def transform_to_tidy(data, start_date):
    tidy_data = []
    for city, city_data in data.items():
        for i, (precip_sum, rain_sum, precip_hours) in enumerate(zip(
            city_data['precipitation_sum'], city_data['rain_sum'], city_data['precipitation_hours']
        )):
            tidy_data.append({
                "City": city,
                "Date": pd.to_datetime(start_date) + pd.Timedelta(days=i),  # Assuming daily data
                "Precipitation Sum": precip_sum,
                "Rain Sum": rain_sum,
                "Precipitation Hours": precip_hours
            })
    return tidy_data

# Define the start dates for each dataset
start_date_hist = "1970-01-01"
start_date_rec = "2010-01-01"

# Transform both datasets and combine them
tidy_data_hist = transform_to_tidy(precipitation_data_hist, start_date_hist)
tidy_data_rec = transform_to_tidy(precipitation_data_rec, start_date_rec)

# Concatenate both datasets into a single DataFrame
all_data = pd.DataFrame(tidy_data_hist + tidy_data_rec)

# test that the dataframe is correctly formatted and all present
print(all_data.head(10))
print(all_data.tail(40))


     City       Date  Precipitation Sum  Rain Sum  Precipitation Hours
0  London 1970-01-01                0.0       0.0                  0.0
1  London 1970-01-02                0.3       0.3                  3.0
2  London 1970-01-03                0.3       0.3                  3.0
3  London 1970-01-04                0.0       0.0                  0.0
4  London 1970-01-05                0.0       0.0                  0.0
5  London 1970-01-06                1.4       0.2                  4.0
6  London 1970-01-07                0.0       0.0                  0.0
7  London 1970-01-08                4.5       0.9                  8.0
8  London 1970-01-09                5.6       5.6                 14.0
9  London 1970-01-10                1.2       1.2                  4.0
         City       Date  Precipitation Sum  Rain Sum  Precipitation Hours
88356  Mumbai 2020-11-22                0.0       0.0                  0.0
88357  Mumbai 2020-11-23                0.0       0.0                

In [7]:
all_data.to_csv("rainfall_data.csv", index=False)


In [6]:
# Calculating the annual rainfall per city per year
all_data['Year'] = all_data['Date'].dt.year
annual_rainfall = all_data.groupby(['City', 'Year']).agg(
    annual_rainfall=('Rainfall Sum', 'sum')
).reset_index()
# Filtering for just 1970 and 2020
rainfall_diff = annual_rainfall[annual_rainfall['Year'].isin([1970, 2020])]
rainfall_pivot = rainfall_diff.pivot(index='City', columns='Year', values='Annual_Rainfall').reset_index()
rainfall_pivot['Rainfall Difference'] = rainfall_pivot[2020] - rainfall_pivot[1970]


KeyError: "Column(s) ['Rainfall Sum'] do not exist"